In [1]:
import json
import os
import numpy as np
import regex as re
import pandas as pd
import tqdm
import json

In [2]:
np.random.seed(0)

In [3]:
train_raw = json.load(open('train_buggy_0.json', 'r'))
print(json.dumps(train_raw[0], indent=4))

{
    "idx": 1,
    "doc": "Expects a height mat as input",
    "code": "@ Override public ImageSource apply ( ImageSource input ) { final int [ ] [ ] pixelMatrix = new int [ 3 ] [ 3 ]  ; int w = input . getWidth ( ) ; int h = input . getHeight ( ) ; int [ ] [ ] output = new int [ h ] [ w ]  ; for ( int j = 1 ; j < ( h - 1 ) ; j ++ ) { for ( int i = 1 ; i < ( w - 1 ) ; i ++ ) { pixelMatrix [ 0 ] [ 0 ] = input . getR ( i - 1 , j - 1 ) ; pixelMatrix [ 0 ] [ 1 ] = input . getRGB ( i - 1 , j ) ; pixelMatrix [ 0 ] [ 2 ] = input . getRGB ( i - 1 , j + 1 ) ; pixelMatrix [ 1 ] [ 0 ] = input . getRGB ( i , j - 1 ) ; pixelMatrix [ 1 ] [ 2 ] = input . getRGB ( i , j + 1 ) ; pixelMatrix [ 2 ] [ 0 ] = input . getRGB ( i + 1 , j - 1 ) ; pixelMatrix [ 2 ] [ 1 ] = input . getRGB ( i + 1 , j ) ; pixelMatrix [ 2 ] [ 2 ] = input . getRGB ( j + 1 , j + 1 ) ; int edge = ( int ) convolution ( pixelMatrix ) ; int rgb = ( ( edge << 16 ) | ( edge << 8 ) ) | edge ; output [ j ] [ i ] = rgb ; } } MatrixSource so

In [4]:
train = []

for row in train_raw:
    train_dict = {}
    train_dict['id'] = 'train-java-{}'.format(row['idx'])
    train_dict['code'] = row['code']
    train_dict['text'] = row['doc']
    train_dict['label'] = row['label']
    train_dict['raw'] = row['raw']
    train_dict['url'] = row['url']
    train.append(train_dict)

In [5]:
test_raw = json.load(open('test_buggy_0.json', 'r'))
print(json.dumps(test_raw[0], indent=4))

{
    "idx": 1,
    "doc": "Makes sure the fast - path emits in order .",
    "code": "protected final void fastPathOrderedEmit ( U value , boolean delayError , Disposable disposable )  { final Observer < ? super V > observer = downstream ; final SimplePlainQueue < U > q = queue ; if ( ( wip . get ( ) == 0 ) && wip . compareAndSet ( 0 , 1 ) ) { if ( q . isEmpty ( ) ) { accept ( observer , value ) ; if ( leave ( - 1 ) == 0 ) { return ; } } else { q . offer ( value ) ; } } else { q . offer ( value ) ; if ( ! enter ( ) ) { return ; } } QueueDrainHelper . drainLoop ( q , value , delayError , disposable , this ) ; }",
    "raw": "",
    "url": "https://github.com/ReactiveX/RxJava/blob/ac84182aa2bd866b53e01c8e3fe99683b882c60e/src/main/java/io/reactivex/internal/observers/QueueDrainObserver.java#L88-L108",
    "label": 1
}


In [6]:
test = []

for row in test_raw:
    test_dict = {}
    test_dict['id'] = 'test-java-{}'.format(row['idx'])
    test_dict['code'] = row['code']
    test_dict['text'] = row['doc']
    test_dict['label'] = row['label']
    test_dict['raw'] = row['raw']
    test_dict['url'] = row['url']
    test.append(test_dict)

In [7]:
np.random.shuffle(train)
np.random.shuffle(test)

In [8]:
java_valid = pd.read_json('../java/data_valid.json')

train_df = pd.DataFrame(train)
valid_df = train_df[train_df['url'].isin(java_valid['url'])]
train_df_left = train_df[~train_df['url'].isin(java_valid['url'])]

print(len(valid_df))
print(len(train_df_left))


9281
83619


In [9]:
train, valid = train_df_left.to_dict('records'), valid_df.to_dict('records')
print(len(train))
print(len(valid))
print(len(test))

83619
9281
5935


In [10]:
print(train[:4])
print(valid[:4])

[{'id': 'train-java-35632', 'code': 'public SDVariable conv1d ( SDVariable input , SDVariable weights , Conv1DConfig conv1DConfig ) { Conv1D conv1D = Conv1D . builder ( ) . inputFunctions ( new SDVariable [ ] { input , input } ) . sameDiff ( sameDiff ( ) ) . config ( conv1DConfig ) . build ( ) ; return conv1D . outputVariable ( ) ; }', 'text': 'Conv1d operation .', 'label': 1, 'raw': 'public SDVariable conv1d(SDVariable input, SDVariable weights, Conv1DConfig conv1DConfig) {\n        Conv1D conv1D = Conv1D.builder()\n                .inputFunctions(new SDVariable[]{input, weights})\n                .sameDiff(sameDiff())\n                .config(conv1DConfig)\n                .build();\n\n        return conv1D.outputVariable();\n    }', 'url': 'https://github.com/deeplearning4j/deeplearning4j/blob/effce52f2afd7eeb53c5bcca699fcd90bd06822f/nd4j/nd4j-backends/nd4j-api-parent/nd4j-api/src/main/java/org/nd4j/autodiff/functions/DifferentialFunctionFactory.java#L274-L282'}, {'id': 'train-java-

In [11]:
with open('data_train.json', 'w') as f:
    json.dump(train, f, indent=2)

with open('data_valid.json', 'w') as f:
    json.dump(valid, f, indent=2)

with open('data_test.json', 'w') as f:
    json.dump(test, f, indent=2)